<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Read-the-data" data-toc-modified-id="Read-the-data-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Read the data</a></span></li><li><span><a href="#Dealing-with-missing-values" data-toc-modified-id="Dealing-with-missing-values-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Dealing with missing values</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Imputation-for-geo-value-with-KNN" data-toc-modified-id="Imputation-for-geo-value-with-KNN-2.0.1"><span class="toc-item-num">2.0.1&nbsp;&nbsp;</span>Imputation for geo value with KNN</a></span></li><li><span><a href="#Filling-missing-values" data-toc-modified-id="Filling-missing-values-2.0.2"><span class="toc-item-num">2.0.2&nbsp;&nbsp;</span>Filling missing values</a></span></li><li><span><a href="#Drop-duplicate-columns" data-toc-modified-id="Drop-duplicate-columns-2.0.3"><span class="toc-item-num">2.0.3&nbsp;&nbsp;</span>Drop duplicate columns</a></span></li></ul></li></ul></li><li><span><a href="#Create-new-features" data-toc-modified-id="Create-new-features-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Create new features</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Check-point:-Save-data-frame" data-toc-modified-id="Check-point:-Save-data-frame-3.0.1"><span class="toc-item-num">3.0.1&nbsp;&nbsp;</span>Check point: Save data frame</a></span></li></ul></li></ul></li></ul></div>

In [1]:
import pandas as pd
import numpy as np
import os

from datetime import datetime

import seaborn as sns
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

# Read the data

- Large dataset will be a challenge. Credited to the implementation from [Faster data loading time in Python](https://www.kaggle.com/c/zillow-prize-1/discussion/37261) and [Reducing DataFrame memory size by ~65%](https://www.kaggle.com/arjanso/reducing-dataframe-memory-size-by-65), loading time is shorten by serializing data instead of the csv files in subsequent runs

- The Training Data dataset contains the log error and transaction dates for 90,275 homes sold during 2016.

- The Property Data dataset contains 58 different features. Unlike the Training Data, this dataset features information on all homes - not just ones that have been sold. We will see that many of the homes in this dataset are missing information.



In [2]:
def load_data():
    # Pickled versions of Data Sets
    train2016_p = './data/train2016_p'
    train2017_p = './data/train2017_p'
    prop2016_p = './data/prop2016_p'
    prop2017_p = './data/prop2017_p'
    sample_p = './data/sample_p'

    # If pickled train2016 exists, load it; else load train_2016_v2.csv to df and pickle it
    if os.path.exists(train2016_p):
        train2016 = pd.read_pickle(train2016_p)
    else:
        # load data to df
        train2016 = pd.read_csv('./data/train_2016_v2.csv',parse_dates=['transactiondate'])
        # create pickled file for storage
        train2016.to_pickle('./data/train2016_p')

    # If pickled train2017 exists, load it; else load train_2017.csv to df and pickle it
    if os.path.exists(train2017_p):
        train2017 = pd.read_pickle(train2017_p)
    else:
        # load data to df
        train2017 = pd.read_csv('./data/train_2017.csv',parse_dates=['transactiondate'])
        # create pickled file for storage
        train2017.to_pickle('./data/train2017_p')

    # If pickled prop2016_p load it; else load properties_2016.csv to df and pickle it
    if os.path.exists(prop2016_p):
        prop2016 = pd.read_pickle(prop2016_p)
    else:
        prop2016 = pd.read_csv('./data/properties_2016.csv')
        prop2016.to_pickle('./data/properties_2016_p')

    # If pickled prop2017_p load it; else load properties_2017.csv to df and pickle it
    if os.path.exists(prop2017_p):
        prop2017 = pd.read_pickle(prop2017_p)
    else:
        prop2017 = pd.read_csv('./data/properties_2017.csv')
        prop2017.to_pickle('./data/properties_2017_p')

    # If pickled sample exists, load it; else load sample_submission.csv to df and pickle it
    if os.path.exists(sample_p):
        sample = pd.read_pickle(sample_p)
    else:
        sample = pd.read_csv('./data/sample_submission.csv')
        sample.to_pickle('./data/sample_p')
    return prop2016, prop2017, train2016, train2017, sample

In [3]:
#Load Datasets
properties_2016, properties_2017, train_2016_v2, train_2017, sample = load_data()

In [4]:
# Print row and columns of each dataset
print(properties_2016.shape)
print(properties_2017.shape)
print(train_2016_v2.shape)
print(train_2017.shape)

(2985217, 58)
(2985217, 58)
(90275, 3)
(77613, 3)


In [5]:
#Merge transaction table with the properties table by parcelid
train_df_2016 = train_2016_v2.merge(properties_2016, on='parcelid', how='left')
train_df_2017 = train_2017.merge(properties_2017, on='parcelid', how='left')

sample['parcelid'] = sample['ParcelId']
del sample['ParcelId']

df = sample.merge(properties_2016, on='parcelid',how='left')
#train_df = pd.concat([train_df_2016, train_df_2017], axis = 0)

print(df.shape)

(2985217, 64)


In [6]:
missing_value= pd.DataFrame(df.isnull().sum(axis=0).sort_values(ascending=False),columns=['missing value'])
missing_value['missing value pct'] = missing_value/ df.shape[0]*100
missing_value

,missing value,missing value pct
storytypeid,2983593,99.945599
basementsqft,2983589,99.945465
yardbuildingsqft26,2982570,99.911330
fireplaceflag,2980054,99.827048
architecturalstyletypeid,2979156,99.796966
typeconstructiontypeid,2978470,99.773986
finishedsquarefeet13,2977545,99.743000
buildingclasstypeid,2972588,99.576949
decktypeid,2968121,99.427311
finishedsquarefeet6,2963216,99.263002


# Dealing with missing values

### Imputation for geo value with KNN 

In [7]:
# Filling the missing 34 location on latitude and longitude
df['latitude'] = df['latitude'].fillna(df['latitude'].mean())
df['longitude'] = df['longitude'].fillna(df['longitude'].mean())

### Filling missing values 

In [8]:
# Most stories are between 1 and 2 as the data suggested
df['numberofstories'][np.isnan(df['numberofstories'])] = np.random.randint(1,3)

# A three-quarter or 3/4 bath, is generally one with a toilet, sink and shower, but not a tub.
df['threequarterbathnbr'] = df['threequarterbathnbr'].fillna(0)

# if Null in garage count it means there are no garages as the sqft for garage 
df['garagecarcnt'] = df['garagecarcnt'].fillna(0)   
df['garagetotalsqft']=  df['garagetotalsqft'].fillna(0)

In [9]:
# From the data dictionary, none is listed as 5 
df['airconditioningtypeid']= df['airconditioningtypeid'].fillna(5)

# From the data dictionary, Other is 19
df['architecturalstyletypeid'] = df['architecturalstyletypeid'].fillna(19)

# From the data dictionary, none is lised as 13
df['heatingorsystemtypeid'] = df['heatingorsystemtypeid'].fillna(13)

In [10]:
# Replace nan value with 0 in the binary object columns
df['hashottuborspa'] = df['hashottuborspa'].apply(lambda x: 1 if x == True else 0)
df['fireplaceflag'] = df['fireplaceflag'].apply(lambda x: 1 if x == True else 0)
df['taxdelinquencyflag'] = df['taxdelinquencyflag'].apply(lambda x: 1 if x=='Y' else 0)

# Fill Nan value with 0 if nan in the binary float columns
for i in ('fireplacecnt', 'poolcnt', 'pooltypeid10','pooltypeid2','pooltypeid7',
          'yardbuildingsqft17','yardbuildingsqft26'):
    df[i] = df[i].fillna(0)

# Fill values with median or number of frequency since 
for i in ('rawcensustractandblock','censustractandblock', 'fips','propertycountylandusecode',
          'propertylandusetypeid',
         'regionidcounty',
         'regionidzip',
         'regionidcity'):
    df[i] = df[i].fillna(df[i].mode()[0])
    
# room count, sqft, and attributes filled with median
for i in ('bathroomcnt','bedroomcnt', 'roomcnt','unitcnt','lotsizesquarefeet', 'calculatedfinishedsquarefeet',
          'buildingqualitytypeid',
          'basementsqft', 'yearbuilt',
          'finishedsquarefeet50'):
    df[i] = df[i].fillna(df[i].median())

# Fill in median for the tax liability
for i in ('assessmentyear','taxvaluedollarcnt', 'taxvaluedollarcnt',
          'landtaxvaluedollarcnt','structuretaxvaluedollarcnt','taxamount'):
    df[i] = df[i].fillna(df[i].median())

# If taxdelinquencyflag is zero, no owing any tax, so does tax delinquency year
df.loc[(df['taxdelinquencyyear'] > 0) & (df['taxdelinquencyyear'].isnull()), 'taxdelinquencyyear'] = df.loc[df['taxdelinquencyflag'] > 0, 'taxdelinquencyyear'].median()
df.loc[(df['taxdelinquencyflag'] == 0), 'taxdelinquencyyear'] = 0

# Fill null values with conditions. If pool count has value and missing the poolsize, fill with median. 
df.loc[(df['poolcnt'] > 0) & (df['poolsizesum'].isnull()), 'poolsizesum'] = df.loc[df['poolcnt'] > 0, 'poolsizesum'].median()
df.loc[(df['poolcnt'] == 0), 'poolsizesum'] = 0

In [11]:
missingcount = df.isnull().sum(axis=0)
missingcount[missingcount>0]

buildingclasstypeid         2972588
calculatedbathnbr            128912
decktypeid                  2968121
finishedfloor1squarefeet    2782500
finishedsquarefeet12         276033
finishedsquarefeet13        2977545
finishedsquarefeet15        2794419
finishedsquarefeet6         2963216
fullbathcnt                  128912
propertyzoningdesc          1006588
regionidneighborhood        1828815
storytypeid                 2983593
typeconstructiontypeid      2978470
dtype: int64

### Drop duplicate columns

In [12]:
# Removing duplicate variables to avoid multicollinearity that will contribute to finishedsquarefeet50 (total area)
# Variables with over 90% missing values is not easy to estimate the correct value may need to be dropped

drop_columns= ['finishedsquarefeet12',
               'finishedsquarefeet13', 
               'finishedsquarefeet15',
               'finishedsquarefeet6',
               'finishedfloor1squarefeet',
               'buildingclasstypeid',
               'calculatedbathnbr',
               'typeconstructiontypeid',
               'fullbathcnt',# same as bathroomcnt
               'decktypeid',# missing too much value as for below attributes
               'storytypeid',
               'typeconstructiontypeid',
               'propertyzoningdesc','regionidneighborhood'
              ]

df = df.drop(columns=drop_columns, axis=1) 

In [13]:
missingcount = df.isnull().sum(axis=0)
missingcount[missingcount>0]

Series([], dtype: int64)

# Create new features

In [15]:
# Get the trasaction month 
#df['N_transaction_month'] = df['transactiondate'].dt.month

# Use the building age instead of year
df['N_building_age'] = [2017 - i for i in df.yearbuilt]

#proportion of living area
df['N_living_area_ratio'] = df['calculatedfinishedsquarefeet']/df['lotsizesquarefeet']

#Ratio of the built structure value to land area
df['N_structure_value_ratio'] = df['structuretaxvaluedollarcnt']/df['landtaxvaluedollarcnt']

# Quality and Room count
df['N_building_quality_room_cnt']= df['buildingqualitytypeid'] * df['roomcnt'] 

# Location
df['N_location'] = df['latitude'] + df['longitude']

# Quality and location
df['N_building_quality_location']= df['buildingqualitytypeid'] * df['N_location'] 

# Number of properties in the region
region_count = df['regionidcounty'].value_counts().to_dict()
df['N_county_count'] = df['regionidcounty'].map(region_count)

# Ratio of tax of property over parcel
df['N_property_tax_value_ratio'] = df['taxvaluedollarcnt']/df['taxamount']

#Does property have a garage, pool or hot tub and AC
df['N_amenities'] = ((df['garagecarcnt']>0) & (df['pooltypeid10']>0) & (df['airconditioningtypeid']!=5))*1 

# Length of time since unpaid taxes
# df['N_unpaid_taxes_length'] = [2017 - i for i in df.taxdelinquencyyear]

#There's 25 different property uses - let's compress them down to 4 categories
df['N_property_uses_Type'] = df.propertylandusetypeid.replace({31 : "Mixed", 46 : "Other", 47 : "Mixed", 246 : "Mixed", 247 : "Mixed", 248 : "Mixed", 260 : "Home", 261 : "Home", 262 : "Home", 263 : "Home", 264 : "Home", 265 : "Home", 266 : "Home", 267 : "Home", 268 : "Home", 269 : "Not Built", 270 : "Home", 271 : "Home", 273 : "Home", 274 : "Other", 275 : "Home", 276 : "Home", 279 : "Home", 290 : "Not Built", 291 : "Not Built" })

In [16]:
from sklearn import preprocessing

le = preprocessing.LabelEncoder()
le.fit(df['propertycountylandusecode'])
df['propertycountylandusecode'] = le.transform(df['propertycountylandusecode'])

le.fit(df['N_property_uses_Type'])
df['N_property_uses_Type'] = le.transform(df['N_property_uses_Type'])

In [17]:
missingcount = df.isnull().sum(axis=0)
missingcount[missingcount>0]

Series([], dtype: int64)

In [19]:
df.dtypes

201610                            int64
201611                            int64
201612                            int64
201710                            int64
201711                            int64
201712                            int64
parcelid                          int64
airconditioningtypeid           float64
architecturalstyletypeid        float64
basementsqft                    float64
bathroomcnt                     float64
bedroomcnt                      float64
buildingqualitytypeid           float64
calculatedfinishedsquarefeet    float64
finishedsquarefeet50            float64
fips                            float64
fireplacecnt                    float64
garagecarcnt                    float64
garagetotalsqft                 float64
hashottuborspa                    int64
heatingorsystemtypeid           float64
latitude                        float64
longitude                       float64
lotsizesquarefeet               float64
poolcnt                         float64


### Check point: Save data frame 

In [20]:
df.to_pickle('./data/df_wip_test')